In [2]:
# Python Lib
import os

# Extended Libs
import numpy as np
import pandas as pd
import json

# Plotting Libs
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.spatial.distance import cdist
from scipy.interpolate import CubicSpline, interp1d

# Choose file:
import tkinter as tk

# 1) Load and format data
AIS data for 3 days at the time the SAR and RF data is received (02-11-2022, 03-11-2022, & 05-11-2022).

A new column with time with dtype datetime64 is generated from the column: bs_ts which has dtype string.

Three columns are renamed: 
- bs_ts = time
- lat = latitude
- lon = longitude

Finally a dict of dataframes is generated from the three days with days being key and dataframes being values.

In [3]:
### Load data & display ###
## AIS data ##
df_ais_0211 = pd.read_csv(filepath_or_buffer ="C:\\Users\\abelt\\OneDrive\\Desktop\\Kandidat\\ais\\ais_110215.csv")
df_ais_0311 = pd.read_csv(filepath_or_buffer ="C:\\Users\\abelt\\OneDrive\\Desktop\\Kandidat\\ais\\ais_110315.csv")
df_ais_0511 = pd.read_csv(filepath_or_buffer ="C:\\Users\\abelt\\OneDrive\\Desktop\\Kandidat\\ais\\ais_1105.csv")
# Compile into a dict #
dfs_ais = {'02-11-2022': df_ais_0211, '03-11-2022': df_ais_0311, '05-11-2022' : df_ais_0511}

# Format time value dtype into datetime dtype #
for df_ais in dfs_ais.values():
    df_ais.rename(columns = {'bs_ts':'time', 'lat':'latitude', 'lon':'longitude'}, inplace = True)
    df_ais['source'] = 'ais'
    df_ais["TimeStamp"] = pd.to_datetime(df_ais['time'])

## Display dict keys and column names in dataframes ##
print('AIS:\n', dfs_ais.keys(), '\n', dfs_ais['02-11-2022'].columns)

AIS:
 dict_keys(['02-11-2022', '03-11-2022', '05-11-2022']) 
 Index(['time', 'callsign', 'cog', 'destination', 'draught', 'eta', 'imo',
       'latitude', 'length', 'longitude', 'mmsi', 'nav_status', 'name', 'sog',
       'shiptype', 'width', 'source', 'TimeStamp'],
      dtype='object')


Choose date for formatting.

Group the AIS data related to the MMSI number so a intepolation can be made for a ship and not just all points.

In [4]:
date_key = '02-11-2022'

ais_mmsi = dfs_ais[date_key].groupby(by=['mmsi'])
print('Statistics, number of data points for the groups: \n', ais_mmsi.size().describe())

mmsi_numbers = list(ais_mmsi.indices.keys())

Statistics, number of data points for the groups: 
 count    2523.000000
mean       25.699564
std        10.001789
min         1.000000
25%        23.000000
50%        29.000000
75%        33.000000
max        41.000000
dtype: float64


# 2) Interpolation of AIS tracks
Using Linear interpolation and Cubic Spline


In [6]:
mmsi_less = []
for mmsi in mmsi_numbers:
    tes = ais_mmsi.get_group(mmsi)
    if len(tes) <= 2:
        mmsi_less.append(mmsi)


splines = {}
for mmsi in mmsi_numbers:
    tes = ais_mmsi.get_group(mmsi)
    
    # Create time series for interpolation
    times = pd.to_numeric(pd.date_range(start=tes['TimeStamp'].min(), end=tes['TimeStamp'].max(), periods=len(tes)*2))
    
    # Linear interpolation (works even for 2 points)
    int_linear = interp1d(pd.to_numeric(tes['TimeStamp']), np.c_[tes['longitude'], tes['latitude']], axis=0, fill_value="extrapolate", kind='linear')

    x_linear, y_linear = int_linear(times).T
    
    # Cubic spline interpolation, only if there are more than 3 points
    if len(tes) > 3:
        spl_cubic = CubicSpline(tes['TimeStamp'], np.c_[tes['longitude'], tes['latitude']])
        x_cubic, y_cubic = spl_cubic(times).T
    else:
        # If there are less than 4 points, use linear interpolation for both
        x_cubic, y_cubic = x_linear, y_linear
    
    # Store the results in the splines dictionary
    splines.update({
        mmsi: {
            'cubic': {'x': x_cubic, 'y': y_cubic}
        }
    })

c:\ProgramData\anaconda3\Lib\site-packages\scipy\interpolate\_interpolate.py:701: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\scipy\interpolate\_interpolate.py:701: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\scipy\interpolate\_interpolate.py:701: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\scipy\interpolate\_interpolate.py:701: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\scipy\interpolate\_interpolate.py:701: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\scipy\interpolate\_interpolate.py:701: Run

# 3)